## Lecture 17 EEP118

In [18]:
#Lecture17.R
#LECTURE 17



# Load the 'pacman' package
library(pacman) 
#packages to use load them now using the pacman "manager"
p_load(dplyr, haven, readr) 
#Another great feature of p_load(): if you try to load a package that is not installed on your machine, p_load() install the package for you, rather than throwing an error. For instance, let’s install and load one final package named ggplot2. 
p_load(ggplot2)



pacman::p_load(lfe, lmtest, haven, sandwich, tidyverse,car)
# lfe for running fixed effects regression
# lmtest for displaying robust SE in output table
# haven for loading in dta files
# sandwich for producing robust Var-Cov matrix
# tidyverse for manipulating data and producing plots
#car for linear hypothesis F tests


In [19]:

#set scientific display off, thank you Roy
options(scipen=999)

#read in a Stata dataset
mydata <- read_dta("Lecture17.dta")
head(mydata)

wage,educ,exper,tenure,nonwhite,female,married,numdep,smsa,northcen,⋯,west,construc,ndurman,trcommpu,trade,services,profserv,profocc,clerocc,servocc
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3.75,2,39,13,0,0,1,0,1,0,⋯,0,0,0,0,0,0,1,0,0,1
2.92,3,51,30,1,0,0,0,1,0,⋯,0,0,0,0,0,1,0,0,0,1
3.51,4,39,15,0,0,1,5,1,0,⋯,1,0,0,0,1,0,0,0,0,0
3.00,4,48,0,1,0,1,0,0,0,⋯,0,0,1,0,0,0,0,0,0,0
3.00,4,36,0,0,0,1,1,0,0,⋯,0,1,0,0,0,0,0,0,0,0
5.20,6,47,13,1,0,1,0,1,0,⋯,0,1,0,0,0,0,0,0,0,0


In [20]:
#generate category education variable
mydata$cateduc=1
mydata$cateduc[mydata$educ==12]<-2
mydata$cateduc[mydata$educ>12]<-3

#summary stats variables
summary(mydata)

      wage             educ           exper           tenure      
 Min.   : 0.530   Min.   : 0.00   Min.   : 1.00   Min.   : 0.000  
 1st Qu.: 3.330   1st Qu.:12.00   1st Qu.: 5.00   1st Qu.: 0.000  
 Median : 4.650   Median :12.00   Median :13.50   Median : 2.000  
 Mean   : 5.896   Mean   :12.56   Mean   :17.02   Mean   : 5.105  
 3rd Qu.: 6.880   3rd Qu.:14.00   3rd Qu.:26.00   3rd Qu.: 7.000  
 Max.   :24.980   Max.   :18.00   Max.   :51.00   Max.   :44.000  
    nonwhite          female          married           numdep     
 Min.   :0.0000   Min.   :0.0000   Min.   :0.0000   Min.   :0.000  
 1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:0.000  
 Median :0.0000   Median :0.0000   Median :1.0000   Median :1.000  
 Mean   :0.1027   Mean   :0.4791   Mean   :0.6084   Mean   :1.044  
 3rd Qu.:0.0000   3rd Qu.:1.0000   3rd Qu.:1.0000   3rd Qu.:2.000  
 Max.   :1.0000   Max.   :1.0000   Max.   :1.0000   Max.   :6.000  
      smsa           northcen         south            

### *1. Does the wage marginal effect of education differ by experience?

In [21]:
#** LECTURE 17 STARTS HERE
##EDUC EXPER
mydata$expeduc=mydata$exper*mydata$educ

six<-lm(wage~female+educ+exper+expeduc, mydata)
summary(six)




Call:
lm(formula = wage ~ female + educ + exper + expeduc, data = mydata)

Residuals:
    Min      1Q  Median      3Q     Max 
-6.6786 -1.8886 -0.5291  1.1104 14.7342 

Coefficients:
             Estimate Std. Error t value            Pr(>|t|)    
(Intercept) -0.489972   1.152289  -0.425               0.671    
female      -2.194000   0.271382  -8.085 0.00000000000000439 ***
educ         0.504442   0.085672   5.888 0.00000000700919782 ***
exper        0.008438   0.040471   0.208               0.835    
expeduc      0.004722   0.003310   1.427               0.154    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3.075 on 521 degrees of freedom
Multiple R-squared:  0.312,	Adjusted R-squared:  0.3067 
F-statistic: 59.06 on 4 and 521 DF,  p-value: < 0.00000000000000022


Step 1: H0: beta_expEduc= 𝛽4  =0   (of the interaction term)
	    H1: 𝛽4  not equal to 0
        
Step 2  𝒕=(𝟎.𝟎𝟎𝟒−𝟎)/(𝟎.𝟎𝟑𝟑)=1.427 = 1.43

Step 3  significance level 1%  526-5 degrees of freedom c=2.58

Step 4 t< c so CANNOT reject null. 

Step 5  We cannot reject at the 1% significance level that marginal effect of experience does not depend on education.

### *2 Ordinal Variables

In [22]:
##5. ordinal variables

mydata$HSDrop<-0
mydata$HSDrop[mydata$cateduc==1]<-1

mydata$HS<-0
mydata$HS[mydata$cateduc==2]<-1

mydata$Col<-0
mydata$Col[mydata$cateduc==3]<-1


#which specification would you choose?

seven1<-lm(wage~female+HSDrop+HS+Col+exper,mydata)
summary(seven1)

seven<-lm(wage~female+HS+Col+exper,mydata)
summary(seven)

eight<-lm(wage~female+cateduc+exper,mydata)
summary(eight)



Call:
lm(formula = wage ~ female + HSDrop + HS + Col + exper, data = mydata)

Residuals:
    Min      1Q  Median      3Q     Max 
-6.7030 -2.0518 -0.5717  1.1494 15.9085 

Coefficients: (1 not defined because of singularities)
            Estimate Std. Error t value             Pr(>|t|)    
(Intercept)  7.66624    0.29043  26.396 < 0.0000000000000002 ***
female      -2.29750    0.28180  -8.153  0.00000000000000267 ***
HSDrop      -3.59468    0.37612  -9.557 < 0.0000000000000002 ***
HS          -1.86299    0.32200  -5.786  0.00000001247892073 ***
Col               NA         NA      NA                   NA    
exper        0.04846    0.01050   4.613  0.00000500002110494 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3.189 on 521 degrees of freedom
Multiple R-squared:  0.2601,	Adjusted R-squared:  0.2544 
F-statistic: 45.78 on 4 and 521 DF,  p-value: < 0.00000000000000022



Call:
lm(formula = wage ~ female + HS + Col + exper, data = mydata)

Residuals:
    Min      1Q  Median      3Q     Max 
-6.7030 -2.0518 -0.5717  1.1494 15.9085 

Coefficients:
            Estimate Std. Error t value             Pr(>|t|)    
(Intercept)  4.07156    0.39408  10.332 < 0.0000000000000002 ***
female      -2.29750    0.28180  -8.153  0.00000000000000267 ***
HS           1.73169    0.37546   4.612  0.00000501946215647 ***
Col          3.59468    0.37612   9.557 < 0.0000000000000002 ***
exper        0.04846    0.01050   4.613  0.00000500002110494 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3.189 on 521 degrees of freedom
Multiple R-squared:  0.2601,	Adjusted R-squared:  0.2544 
F-statistic: 45.78 on 4 and 521 DF,  p-value: < 0.00000000000000022



Call:
lm(formula = wage ~ female + cateduc + exper, data = mydata)

Residuals:
    Min      1Q  Median      3Q     Max 
-6.6861 -2.0471 -0.6054  1.1574 15.9244 

Coefficients:
            Estimate Std. Error t value             Pr(>|t|)    
(Intercept)  2.23966    0.52180   4.292  0.00002109145187591 ***
female      -2.30608    0.27890  -8.268  0.00000000000000113 ***
cateduc      1.80437    0.18522   9.742 < 0.0000000000000002 ***
exper        0.04837    0.01049   4.612  0.00000501591449962 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3.186 on 522 degrees of freedom
Multiple R-squared:   0.26,	Adjusted R-squared:  0.2557 
F-statistic: 61.13 on 3 and 522 DF,  p-value: < 0.00000000000000022


Use adjusted R square to choose between seven1 , seven, or eight

I would choose the eight because adjusted R squared is 0.2557 greater than the other adjusted R squared


### *3. ARE the wage equations the same for males and females?

In [23]:
###6. Are the wage equations for male and female the same?


#generate the interactions for the unrestricted model
mydata$femexper<-mydata$female*mydata$exper
mydata$femeduc<-mydata$female*mydata$educ


#do an F test

#unrestricted model 
#𝑤𝑎𝑔𝑒=𝛽_0+𝛽_1  𝑓𝑒𝑚𝑎𝑙𝑒+𝛽_2  𝑒𝑥𝑝𝑒𝑟+ 𝛽_3 𝑒𝑑𝑢+ 𝛽_4  𝑓𝑒𝑚𝑎𝑙𝑒∗𝑒𝑥𝑝𝑒𝑟+ 𝛽_5  𝑓𝑒𝑚𝑎𝑙𝑒∗𝑒𝑑𝑢 +𝑣

twelve<-lm(wage~educ+exper+female+femeduc+femexper, mydata)
summary(twelve)
#get SSR unrestricted

#restricted model 
#𝑤𝑎𝑔𝑒=𝛽_0+𝛽_2  𝑒𝑥𝑝𝑒𝑟+ 𝛽_3 𝑒𝑑𝑢 +𝑣
nine<-lm(wage~educ+exper,mydata)
summary(nine)
#get SSR restricted

#compute the F test stat and compare to the F critical value



Call:
lm(formula = wage ~ educ + exper + female + femeduc + femexper, 
    data = mydata)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.6728 -1.6624 -0.5504  1.0819 14.6058 

Coefficients:
            Estimate Std. Error t value             Pr(>|t|)    
(Intercept) -4.15899    0.97703  -4.257   0.0000246092392071 ***
educ         0.72550    0.06540  11.094 < 0.0000000000000002 ***
exper        0.11280    0.01455   7.755   0.0000000000000471 ***
female       2.58540    1.44902   1.784               0.0750 .  
femeduc     -0.24940    0.10264  -2.430               0.0154 *  
femexper    -0.09473    0.02042  -4.639   0.0000044278933186 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3.018 on 520 degrees of freedom
Multiple R-squared:  0.3383,	Adjusted R-squared:  0.332 
F-statistic: 53.18 on 5 and 520 DF,  p-value: < 0.00000000000000022



Call:
lm(formula = wage ~ educ + exper, data = mydata)

Residuals:
    Min      1Q  Median      3Q     Max 
-5.5532 -1.9801 -0.7071  1.2030 15.8370 

Coefficients:
            Estimate Std. Error t value             Pr(>|t|)    
(Intercept) -3.39054    0.76657  -4.423       0.000011846649 ***
educ         0.64427    0.05381  11.974 < 0.0000000000000002 ***
exper        0.07010    0.01098   6.385       0.000000000378 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3.257 on 523 degrees of freedom
Multiple R-squared:  0.2252,	Adjusted R-squared:  0.2222 
F-statistic: 75.99 on 2 and 523 DF,  p-value: < 0.00000000000000022


SSR unrestricted from above output ot regression twelve is 𝑆𝑆𝑅=𝑠^2 (𝑛−𝑘)=3.018∗3.018∗520= 4736.33

SSR restricted from above output of regression nine is 𝑆𝑆𝑅𝑟=S𝑟^2 (𝑛−𝑘_𝑟 )=3.257∗3.257∗523= 5548

𝑭=((𝑺𝑺𝑹_𝑹−𝑺𝑺𝑹_𝒖)/𝒒)/(𝑺𝑺𝑹𝒖/(𝑵−𝒌−𝟏))= ((𝟓𝟓𝟒𝟖−𝟒𝟕𝟑𝟔.𝟑𝟑)/𝟑)/((𝟒𝟕𝟑𝟔.𝟑𝟑)/(𝟓𝟐𝟔−𝟓−𝟏))=𝟐𝟗.𝟔𝟓

p value of that F is 2.2e-16 ***  the p value is basically zero, has three stars

so we reject the null of male and female having the same wage equation at the 1% significance, at 5% and at 10%. 


Alternatively you could get the critical F for 1percent significance for an F with 3 and 520 degrees of freedom

3 restrictions and N-k for the unrstricted model is 520.



In [24]:
#In R you can do the F test above by estimating twelve and then
#testing the joint null that the  female femeduc femexper are all zero (female=femeduc=femexper=0)

twelve<-lm(wage~educ+exper+female+femeduc+femexper, mydata)
summary(twelve)

linearHypothesis(twelve, c("female=0", "femeduc=0","femexper=0"))


Call:
lm(formula = wage ~ educ + exper + female + femeduc + femexper, 
    data = mydata)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.6728 -1.6624 -0.5504  1.0819 14.6058 

Coefficients:
            Estimate Std. Error t value             Pr(>|t|)    
(Intercept) -4.15899    0.97703  -4.257   0.0000246092392071 ***
educ         0.72550    0.06540  11.094 < 0.0000000000000002 ***
exper        0.11280    0.01455   7.755   0.0000000000000471 ***
female       2.58540    1.44902   1.784               0.0750 .  
femeduc     -0.24940    0.10264  -2.430               0.0154 *  
femexper    -0.09473    0.02042  -4.639   0.0000044278933186 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3.018 on 520 degrees of freedom
Multiple R-squared:  0.3383,	Adjusted R-squared:  0.332 
F-statistic: 53.18 on 5 and 520 DF,  p-value: < 0.00000000000000022


,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,523,5548.160,NA,NA,NA,NA
2,520,4737.783,3,810.3771,29.64791,0.00000000000000001043344


The F is 29.65. as before

We see that the p value of that F is 0.000000000000000 *** the p value is basically zero, has three stars

so we reject the null of male and female having the same wage equation at the 1% significance, at 5% and at 10%.

Alternatively you could get the critical F for 1percent significance for an F with 3 and 520 degrees of freedom

3 restrictions and N-k for the unrstricted model is 520.

#### The Chow Test - should the model be run together males and females or as separate models?

In [ ]:
#separate equations:
ten<-lm(wage~educ+exper, mydata[which(mydata$female==1),])
summary(ten)

eleven<-lm(wage~educ+exper, mydata[which(mydata$female==0),])
summary(eleven)

#one single equation, the restricted model
nine<-lm(wage~educ+exper,mydata)
summary(nine)

#or  Chow test (new but same idea as F test)
#Model assuming same equation
#Model with two separate equations


#the Chow test is also an F test

𝑭=( (𝑺𝑺𝑹_𝑹−(𝑺𝑺𝑹_𝒇+𝑺𝑺𝑹𝒎)) / (𝒌+𝟏))  / ((𝑺𝑺𝑹𝒇+𝑺𝑺𝑹𝒎) / (𝑵𝒇−𝒌−𝟏+𝑵𝒎−𝒌−𝟏) )

In [ ]:
#run restrictec model, get SSR_r
#one single equation, the restricted model
nine<-lm(wage~educ+exper,mydata)
summary(nine)

As before SSR of nine, SRR_R = 5548


In [ ]:
#separate equations:
ten<-lm(wage~educ+exper, mydata[which(mydata$female==1),])
summary(ten)

eleven<-lm(wage~educ+exper, mydata[which(mydata$female==0),])
summary(eleven)

#and get SSR_f  and SSR_m

From Residual standard error of female in regression ten : 2.264 on 249 degrees of freedom ---> SSR_f=2.262*2.262*249= 𝟏𝟐𝟕𝟓.𝟕𝟑

From residual stad error of eleven, for males, Residual standard error: 3.574 on 271 degrees of freedom, SSR_m=3.574*3.574*271= 3462.045

Then 4737.775 =SSRf+SSRm

Substituting into the Chow F formula:

F= ((5548.159−1275.73+3462.045)/(2+1))/((1275.73+3462.045)/(252−2−1+274−2−1))=𝟐𝟗.𝟔𝟓

Given this F > 2.6 =the critical value for 5% for an F distrib with degrees of freedom (3,520)
So we reject the null that the equation should be estimated pooled into the same equation.


# Evolution of Wage gap over time

We will now investigate the evolution of the gender wage gap over time

We will use a survey dataset for 1978, a cross section of responses for that year 1978, N=550

We will also use a survey dataset for 1985, a cross section of responses for that year 1985, N=534


When we combine the two cross sections in the analysis we are pooling two cross sections, or we are doing a pooled cross section analysis

Dataset Lecture17_CPS78_85.dta

In [26]:
#ADDITIONAL DATASET
#***use now 1978 and 1985 CPS pooled cross sections
#read in this new data set
mydata2 <- read_dta("Lecture17_CPS78_85.DTA")
head(mydata2)


#the original variable was log wage, lets get the wage level to use in the analysis below:
mydata2$wage<-exp(mydata2$lwage)

educ,south,nonwhite,female,married,exper,expersq,union,lwage,age,year,y85,y85fem,y85educ,y85union
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
12,0,0,0,0,8,64,0,1.2150,25,78,0,0,0,0
12,0,0,1,1,30,900,1,1.6094,47,78,0,0,0,0
6,0,0,0,1,38,1444,1,2.1401,49,78,0,0,0,0
12,0,0,0,1,19,361,1,2.0732,36,78,0,0,0,0
12,0,0,0,1,11,121,0,1.6490,28,78,0,0,0,0
8,0,0,0,1,43,1849,0,1.7148,56,78,0,0,0,0


####   Create real wage


Go to CPI calculator form the BLS:
  https://www.bls.gov/data/inflation_calculator.htm

The Consumer Price Index (CPI)  is the average price
for 200 items with base in a certain year. 
1 dollar in 1978 equals 1.69 dollars in 1985.

Or get the CPI index for 1978 and for 1985 with base 
year 1982

http://www.usinflationcalculator.com/inflation/consumer-price-index-and-annual-percent-changes-from-1913-to-2008/

CPI 1978=  CPI 1978= 65.2 and CPI for 1985= 107.6

so, 

In the code cell  that comes next, we generate a new variable rwage=wage

then replace rwage=wage*107.6/65.2 if year==78 ---- for the year 78

The above command puts everything in 1985 dollars so we can compare now and combine 1978 data with 1985 data. Note that we could have put all into 1978 dollars off course, instead…


In [29]:
#create real wage then in 1985 dollars

mydata2$rwage85=mydata2$wage
#replace rwage=wage*107.6/65.2 if year==78
mydata2$rwage[mydata2$year==78]<-mydata2$wage[mydata2$year==78]*107.6/65.2
#label variable rwage``average real hourly earnings in 1985 dollars''



### *5. How does the gender wage gap evolve over time?

In [30]:
reg17one<-lm(rwage~educ+exper+union+female+y85,mydata2)
summary(reg17one)

#THE REGRESSION ABOVE, with the output below,  DOES NOT HELP US TO LOOK AT FEMALE WAGE DIFFERENCE 
#FROM MALE OVER TIME, JUST REAL WAGES OVER TIME  WITH THE  YR85 DUMMY
#and also look at female versus male one average with the FEMALE dummy




Call:
lm(formula = rwage ~ educ + exper + union + female + y85, data = mydata2)

Residuals:
    Min      1Q  Median      3Q     Max 
-12.194  -2.602  -0.525   1.761  38.479 

Coefficients:
            Estimate Std. Error t value             Pr(>|t|)    
(Intercept) -3.24870    0.79830  -4.070          0.000050548 ***
educ         0.90308    0.05319  16.980 < 0.0000000000000002 ***
exper        0.12948    0.01124  11.521 < 0.0000000000000002 ***
union        1.62324    0.32307   5.024          0.000000591 ***
female      -2.62140    0.27572  -9.507 < 0.0000000000000002 ***
y85         -0.88095    0.27257  -3.232              0.00127 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.416 on 1078 degrees of freedom
Multiple R-squared:  0.304,	Adjusted R-squared:  0.3008 
F-statistic: 94.17 on 5 and 1078 DF,  p-value: < 0.00000000000000022


What does the coefficient estimated for y85 mean?


After controlling for gender, education, experience, union, the estimated betahat for dummy for year 1985 is -0.88. 

what does this mean?


AVERAGE WAGES IN 85 RELATIVE TO NOT 85 (YEAR 78) ARE SIGNIF LOWER BY 88 CENTS/HOUR



In [31]:
#SO HOW CAN WE GET AT FEMALE REAL WAGE EVOLUTION FROM 78 TO 85?
#-----  INTERACT YR85 WITH FEMALE DUMMY

mydata2$y85female=mydata2$y85*mydata2$female
reg17two<-lm(rwage~educ+exper+union+female+y85+y85female,mydata2)
summary(reg17two)


Call:
lm(formula = rwage ~ educ + exper + union + female + y85 + y85female, 
    data = mydata2)

Residuals:
    Min      1Q  Median      3Q     Max 
-12.355  -2.608  -0.580   1.790  38.205 

Coefficients:
            Estimate Std. Error t value             Pr(>|t|)    
(Intercept) -3.04096    0.80713  -3.768             0.000174 ***
educ         0.90295    0.05314  16.992 < 0.0000000000000002 ***
exper        0.12771    0.01128  11.324 < 0.0000000000000002 ***
union        1.63265    0.32285   5.057  0.00000050022661594 ***
female      -3.08856    0.39124  -7.894  0.00000000000000715 ***
y85         -1.26580    0.35572  -3.558             0.000389 ***
y85female    0.92200    0.54826   1.682             0.092923 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.412 on 1077 degrees of freedom
Multiple R-squared:  0.3058,	Adjusted R-squared:  0.302 
F-statistic: 79.08 on 6 and 1077 DF,  p-value: < 0.00000000000000022


What do you see in the output above?

In 1978 females earn less -3.08 / hour on average (yr85=0)


In 1985 females earn -3.08+0.92199= -2.158/ hour on average (yr85=1)



The conditional female effect on real wages is 
(𝝏 𝒘𝒂𝒈𝒆)/(𝝏 𝒇𝒆𝒎𝒂𝒍𝒆)  = 𝛽_4 + 𝛽_6 𝑦𝑟85


= -3.08 + 0.92199 = -2.158

Conclusion, the gender gap has shrunk, before in 1978 it was -3.08 dollars/hour and in 1985 it is now -2.158


Daily Assignm: how would you generate a new variable and test whether the marginal increase in wage due
to being in a union has changed over time also from 1978 to 1985 in terms of real wage marginal effects? 



Conclusion, the gender gap has shrunk, before in 1978 it was -3.08 (women earned less 3)  dollars/hour and in 1985 it is now -2.158 (women earn now only 2.15 less)

Looking at 1996 to 2006 I found the gender gap widening, men earned more 4.1 dollars/ hour in 1996 and in 2006 men earn more 5.4 dollars/ hour… curious about more recent data, 2019, 2016 real wage gender gap…



# RECAP lecture 16 and 17 - for international's Women Day

Lecture 16:

average wage men = 7.1  average wage women=4.6   difference=2.5  dollars different average wages

on average women earn 25% less than men

looking at the distribution, missing for women mass in the 20-40 $ an hour range

wages are lower for women even after controlling for education, experience, sector of activity, and union participation. Results are Very robust

Did not show but could show by interacting female and HS, female and Col dummies that the wage differential between men and women is at all levels of education, but largest difference is for college education level

Lecture 17: 

From 1978 to 85 real wages dropped. 

Females earn less than men in 1978 and the real wage gap decreased, in that in 1985 women earn less 2.15 dollars/ hour in real wages, whereas in 85 they earned 3 dollars less

Not shown with data but, over time, 1996-2006, there is not real change at all for wages for men (coeff on yr2006) but a decline for real wages for women of 1.33 dollars.

Interesting to see what is happening in 2016/ and 2019 even…

